In [2]:
%load_ext autoreload
%autoreload 2

from clearmap_viz.graph_utils import load_graph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-05-19 10:01:28 [INFO] ClearMap not accessible. Appending ClearMap local path.


In [3]:
g = load_graph("../data/graphs/Ge1364C_graph_v03.gt")

In [4]:
g

Graph[243966·][312292-]|G|

In [5]:
g.compute_dfs()

Graph[243966·][312292-]|G|

In [7]:
from ClearMap.Analysis.Graphs.GraphRendering import interpolate_edge_geometry

In [10]:
interpolate_edge_geometry(g, processes=1, verbose=True)

ValueError: Invalid inputs.

In [6]:
g.plot_components()

TypeError: 'NoneType' object is not subscriptable

Original Traceback (most recent call last):
  File "/Users/edmz/code/ChristophKirst/ClearMap2/ClearMap/ParallelProcessing/ParallelTraceback.py", line 32, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/edmz/code/ChristophKirst/ClearMap2/ClearMap/Analysis/Graphs/GraphRendering.py", line 296, in _parallel_interpolate
    start,end   = smm.get(indices_hdl)[i];
                  ~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subscriptable
